In [ ]:
!git clone https://github.com/kirilllzaitsev/calibrated-backprojection-network
!cd calibrated-backprojection-network && pip install . && cd ..

In [1]:
import rosbag
import numpy as np
import os
import cv2
from pathlib import Path
from kbnet import data_utils
import yaml
%matplotlib inline

In [ ]:
do_prep_in_kbnet_format = True
do_crop=False

In [3]:
subsample_rosbag_path="./kirill_data_test.bag"

if do_prep_in_kbnet_format:
    base_output_dir='./kbnet_format_subsample'
else:
    base_output_dir='./kitti_format_subsample'
base_data_dir="./data/camera/point_cloud_colorizer_ros/depth_image_camera_2"

validity_map_dir=f"{base_output_dir}/validity_map"
img_dir=f"{base_output_dir}/image"
cam_dir=f"{base_output_dir}/intrinsics"
paths_dir=f"{base_output_dir}/paths"
depth_output_dir=f'{base_output_dir}/sparse_depth'
for dir in [validity_map_dir, img_dir, cam_dir, paths_dir, depth_output_dir]:
    os.makedirs(dir, exist_ok=True)

In [4]:
bag = rosbag.Bag(subsample_rosbag_path)
info_dict = yaml.safe_load(bag._get_yaml_info())

In [5]:
info_dict

{'path': './kirill_data_test.bag',
 'version': 2.0,
 'duration': 1600.432037,
 'start': 1680764280.802401,
 'end': 1680765881.234438,
 'size': 21148188278,
 'messages': 15724,
 'indexed': True,
 'compression': 'lz4',
 'uncompressed': 97817475536,
 'compressed': 21144547291,
 'types': [{'type': 'sensor_msgs/Image',
   'md5': '060021388200f6f0f447d0fcd9c64743'}],
 'topics': [{'topic': '/point_cloud_colorizer_ros/depth_image_camera_2',
   'type': 'sensor_msgs/Image',
   'messages': 7862,
   'frequency': 5.0019},
  {'topic': '/point_cloud_colorizer_ros/point_cloud_overlaid_camera_2',
   'type': 'sensor_msgs/Image',
   'messages': 7862,
   'frequency': 5.0018}]}

EXTRACTION

In [47]:
sparse_dm_topic, image_topic = [
    "/point_cloud_colorizer_ros/depth_image_camera_2",
    "/point_cloud_colorizer_ros/point_cloud_overlaid_camera_2",
]

SAMPLE

In [ ]:
import typing as t
from attr import dataclass


@dataclass
class RosbagSample:
    topic: str
    msg: t.Any
    ts: str

In [48]:
bag_sample_raw = next(iter(bag.read_messages(topics=[sparse_dm_topic])))
bag_sample = RosbagSample(*bag_sample_raw)

In [49]:
info_dict

{'path': '/media/master/wext/cv_data/sensor_data/forest/subsample/first_5_minute_compressed.bag',
 'version': 2.0,
 'duration': 329.9856,
 'start': 1680764280.802401,
 'end': 1680764610.788001,
 'size': 21487811932,
 'messages': 3454,
 'indexed': True,
 'compression': 'none',
 'types': [{'type': 'sensor_msgs/Image',
   'md5': '060021388200f6f0f447d0fcd9c64743'}],
 'topics': [{'topic': '/point_cloud_colorizer_ros/depth_image_camera_2',
   'type': 'sensor_msgs/Image',
   'messages': 1727,
   'frequency': 5.0108},
  {'topic': '/point_cloud_colorizer_ros/point_cloud_overlaid_camera_2',
   'type': 'sensor_msgs/Image',
   'messages': 1727,
   'frequency': 5.0076}]}

DEPTH

In [18]:
def map_values_to_different_range(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = (value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [30]:
from tqdm import tqdm

forest_max_depth = 40
kitti_max_depth = 80

total = 1000
for index, (bag_sample) in tqdm(
    enumerate(bag.read_messages(topics=[sparse_dm_topic])),
    total=total,
):
    if index >= total:
        break
    bag_sample = RosbagSample(*bag_sample)
    im=(np.frombuffer(bag_sample.msg.data, np.float32)).reshape(bag_sample.msg.height, bag_sample.msg.width, -1)
    time_str = f"{bag_sample.msg.header.stamp.to_sec():.6f}"
    filename = f"{time_str[0:18]}.png"
    output_img_path = f"{depth_output_dir}/{filename}"
    dm = im[:,:,0]
    if do_crop:
        dm=dm[300:300+352,100:100+1216]
    dm=map_values_to_different_range(dm, 0, forest_max_depth, 0, kitti_max_depth)
    data_utils.save_depth(dm, output_img_path)

100%|██████████| 1000/1000 [00:30<00:00, 32.91it/s]


VALIDITY MAP

In [12]:

for path in Path(base_data_dir).iterdir():
    _, validity_map = data_utils.load_depth_with_validity_map(path)

    validity_map_output_path=f"{validity_map_dir}/{path.name}"
    data_utils.save_validity_map(validity_map, validity_map_output_path)

IMAGE

In [13]:
from pathlib import Path
base_data_dir="./data/camera/point_cloud_colorizer_ros/point_cloud_overlaid_camera_2"

for path in tqdm(Path(base_data_dir).iterdir(), total=total):
    image = cv2.imread(str(path))
    if do_crop:
        image=image[300:300+352,100:100+1216,:]
    if do_prep_in_kbnet_format:
        image = np.concatenate([image, image, image], axis=1)
    cv2.imwrite(f"{img_dir}/{path.name}", image)

1001it [02:23,  6.98it/s]                          


INTRINSICS

In [14]:
import pandas as pd

cam_info = pd.read_csv("./alphasense_driver_ros-cam4-color-camera_info.csv")
K_idx_range=cam_info.columns.to_list().index("K_0"), cam_info.columns.to_list().index("K_8")
P_idx_range=cam_info.columns.to_list().index("P_0"), cam_info.columns.to_list().index("P_11")
R_idx_range=cam_info.columns.to_list().index("R_0"), cam_info.columns.to_list().index("R_8")
num_subsamples = 974
subsample = cam_info[cam_info["header.frame_id"].str.contains("cam4")].iloc[:num_subsamples]
filenames = subsample.Time.apply(
    lambda time_str: f"{str(time_str)[0:18]}"
)

intrinsics = subsample.iloc[:,K_idx_range[0]:K_idx_range[1]+1].values.reshape(-1, 3,3)

In [15]:
# note: we will take the center crop of the images during augmentation
# that changes the optical centers, but not focal lengths
if do_crop:
    orig_size=(1440, 1080)
    # orig_size=(1242, 375)
    new_size=(1216, 352)
    intrinsics[:, 0, 2] = (
        intrinsics[:, 0, 2] - abs(orig_size[0]-new_size[0])/2
    )
    intrinsics[:, 1, 2] = (
        intrinsics[:, 1, 2] - abs(orig_size[1]-new_size[1])/2
    )

In [16]:

for path, intrinsic in tqdm(zip(filenames, intrinsics)):
    full_path = f"{cam_dir}/{path}.npy"
    np.save(full_path, intrinsic)

974it [00:00, 4504.67it/s]


FORM PATHS

In [17]:
from pathlib import Path

for entity in ["sparse_depth", "validity_map", "image", "intrinsics"]:
    base_data_dir=f"{base_output_dir}/{entity}"
    with open(f"{paths_dir}/{entity}.txt", "w") as f:
        for path in Path(base_data_dir).iterdir():
            entity_output_path=f"{base_output_dir}/{entity}/{path.name}"
            f.write(f"{entity_output_path}\n")